In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install easyocr
!apt-get install poppler-utils
!pip install pdf2image
!pip install python-poppler
!pip uninstall opencv-python-headless==4.5.5.62
!pip install opencv-python-headless==4.5.2.52

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import easyocr
from pylab import rcParams
from pdf2image import convert_from_path
import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import urllib.request
from PIL import Image


In [ ]:
ds = pd.read_csv("/content/Data Engineer Task - Data Engineer Task.csv")

In [ ]:
files = ds.iloc[:,0].values
files = list(files)

In [ ]:
edit_links=[]
for url in files:
  if(url.endswith('.pdf')):
    edit_links.append(url)
    
  else:
    response = requests.get(url)
    soup= BeautifulSoup(response.text, "html.parser")     
    link = soup.select("a[href$='.pdf']")[0]
  
    filename = os.path.join(link['href'].split('/')[-1])
    edit_links.append(urljoin(url,link['href']))

In [ ]:
folder_location = r'/content/PDF'
if not os.path.exists(folder_location):os.mkdir(folder_location)

In [ ]:
for i in range(len(edit_links)):
  print(edit_links[i])
 

  filename = os.path.join(folder_location,str(i)+'.pdf')
  with open(filename, 'wb') as f:
      f.write(requests.get(edit_links[i]).content)
  

In [ ]:
reader = easyocr.Reader(['en','hi','mr'])

In [ ]:
res = []
final_text = ""

In [ ]:
for i in range(len(edit_links)):
  final_text=''
  pages = convert_from_path(os.path.join(folder_location,str(i)+'.pdf'))
  for page in pages:
    page.save('page.png')
    img_text = reader.readtext('page.png')


    for _, text, __ in img_text: # _ = bounding box, text = text and __ = confident level
        final_text += " "
        final_text += text
  unit = {    
          "page-url": files[i],
          "pdf-url": edit_links[i],
          "pdf-content":final_text
   }
  res.append(unit)


In [ ]:
with open('/content/drive/MyDrive/submission.json', 'w', encoding='utf8') as json_file:
    j.dump(res, json_file, ensure_ascii=False)